In [ ]:
!pip install sent2vec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 27.7 MB/s 
     |████████████████████████████████| 6.6 MB 52.6 MB/s 
     |████████████████████████████████| 596 kB 43.2 MB/s 
     |████████████████████████████████| 86 kB 6.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np
import re
import gensim
#import contractions
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import gensim
from gensim.utils import simple_preprocess
from sklearn.cluster import KMeans
from gensim.models import Word2Vec
from sklearn import cluster
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from nltk.cluster import KMeansClusterer, euclidean_distance
import nltk
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import seaborn as sns
from wordcloud import WordCloud
from nltk.tokenize import wordpunct_tokenize
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
#import flair
from scipy.spatial import distance
#from simpletransformers.language_representation import RepresentationModel
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
#import umap.umap_ as umap
from sklearn.decomposition import PCA
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import silhouette_score
from wordcloud import WordCloud
#import texthero as hero
from sklearn import metrics
from scipy import spatial
from sent2vec.vectorizer import Vectorizer
%matplotlib inline

In [ ]:
!pip install -U sentence-transformers
#!pip install "gensim==3.8.1"
#!pip install texthero
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 4.0 MB/s 
     |████████████████████████████████| 1.2 MB 14.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=c6a9ef574a8b85d213171224b0db6e91ed1fa4969db6614f390b9cbd021c83e4
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 106 kB 30.0 MB/s 
     |████████████████████████████████| 287 kB 51.8 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_train=pd.read_csv('/content/drive/MyDrive/BBC News Train.csv')
data_train.drop(['ArticleId'],inplace=True,axis=1)
data_train.head()

,Text,Category
0,worldcom ex-boss launches defence lawyers defe...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in $168m payout eighteen former e...,business


In [ ]:
list_text=data_train['Text'].to_list()

In [ ]:
length=data_train['Text'].apply(len).mean()
print("Average length of Text",length)

Average length of Text 2233.461744966443


In [ ]:
def tagged_document(list_of_list_of_words):
  for i,list_of_words in enumerate(list_of_list_of_words):
    yield gensim.models.doc2vec.TaggedDocument(list_of_words,[i])

data_for_training=list(tagged_document(list_text))    

In [ ]:
model=gensim.models.doc2vec.Doc2Vec(vector_size=2240)

In [ ]:
model.build_vocab(data_for_training)

#Features from Doc2vec

In [ ]:
features=[]
for i in range(len(list_text)):
  features.append(model.infer_vector(list_text[i].split()))

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(5, n_init=10, random_state=42)
kmeans.fit(features)
data_train['cluster'] = kmeans.labels_

In [ ]:
cluster_to_category = {}
for cat in data_train['Category'].unique():
    mark = data_train['Category'] == cat
    top = data_train[mark]['cluster'].value_counts().head(1)
    count = top.values[0]
    cluster = top.index[0]
    print(f"{cat}:\n Top cluster number: {cluster}, Number of samples: {count}")
    cluster_to_category[cluster] = cat

print("\nMap cluster number to category:")
cluster_to_category

business:
 Top cluster number: 2, Number of samples: 136
tech:
 Top cluster number: 2, Number of samples: 91
politics:
 Top cluster number: 2, Number of samples: 113
sport:
 Top cluster number: 2, Number of samples: 122
entertainment:
 Top cluster number: 2, Number of samples: 101

Map cluster number to category:


{2: 'entertainment'}

In [ ]:
data_train['clustered_category'] = data_train['cluster'].map(cluster_to_category)

In [ ]:
print('Overall accuracy of clustered categories:', np.mean(data_train['Category'] == data_train['clustered_category']))

Overall accuracy of clustered categories: 0.06778523489932886


In [ ]:
from sklearn.metrics import silhouette_score
print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(features,data_train['cluster']))

Silhouette Coefficient: 0.000


#Doc2Vec + Preprocessed Text + Kmeans


In [ ]:
import pandas as pd
import numpy as np
import re
import gensim
import contractions
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import gensim
from gensim.utils import simple_preprocess
from sklearn.cluster import KMeans
from gensim.models import Word2Vec
from sklearn import cluster
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from nltk.cluster import KMeansClusterer, euclidean_distance
import nltk
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import seaborn as sns
from wordcloud import WordCloud
from nltk.tokenize import wordpunct_tokenize
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
nltk.download('stopwords') 
nltk.download('wordnet')
nltk.download('omw-1.4')
data_train=pd.read_csv('/content/drive/MyDrive/BBC News Train.csv')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


In [ ]:
STOPWORDS = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

In [ ]:
data_train.drop(['ArticleId'],axis=1,inplace=True)

In [ ]:
def expand_contractions(text):
    expanded_words = []    
    for word in text.split():
        expanded_words.append(contractions.fix(word))   
    expanded_text = ' '.join(expanded_words)
    return expanded_text
 
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
def remove_digits(text):
    return " ".join([word for word in str(text).split() if not(word.isdigit())])
def remove_shorttokens(text):
    return " ".join([word for word in str(text).split() if len(word)>1])
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
#convert it to string
data_train["Text"]=data_train["Text"].astype(str)
#Convert to lower and strip
data_train["Text"]=data_train["Text"].str.lower().str.strip()
#apply contractions
data_train["Text"]=data_train["Text"].apply(lambda x:expand_contractions(x))
#remove punctuations
data_train["Text"]=data_train["Text"].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))
#remove stop words
data_train["Text"]=data_train["Text"].apply(lambda x: remove_stopwords(x))
#Lemmatize the sentence
data_train["Text"]=data_train["Text"].apply(lambda text: lemmatize_words(text))
data_train["Text"]=data_train["Text"].apply(lambda text:remove_digits(text))
data_train["Text"]=data_train["Text"].apply(lambda text:remove_shorttokens(text))

In [ ]:
list_text=data_train['Text'].to_list()

In [ ]:
length=data_train['Text'].apply(len).mean()
print("Average length of Text",length)

Average length of Text 1501.989932885906


In [ ]:
def tagged_document(list_of_list_of_words):
  for i,list_of_words in enumerate(list_of_list_of_words):
    yield gensim.models.doc2vec.TaggedDocument(list_of_words,[i])

data_for_training=list(tagged_document(list_text))    

In [ ]:
model1=gensim.models.doc2vec.Doc2Vec(vector_size=1600)

In [ ]:
model1.build_vocab(data_for_training)

In [ ]:
features=[]
for i in range(len(list_text)):
  features.append(model1.infer_vector(list_text[i].split()))

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(5, n_init=10, random_state=42)
kmeans.fit(features)
data_train['cluster'] = kmeans.labels_

In [ ]:
cluster_to_category = {}
for cat in data_train['Category'].unique():
    mark = data_train['Category'] == cat
    top = data_train[mark]['cluster'].value_counts().head(1)
    count = top.values[0]
    cluster = top.index[0]
    print(f"{cat}:\n Top cluster number: {cluster}, Number of samples: {count}")
    cluster_to_category[cluster] = cat

print("\nMap cluster number to category:")
cluster_to_category

business:
 Top cluster number: 1, Number of samples: 97
tech:
 Top cluster number: 2, Number of samples: 64
politics:
 Top cluster number: 1, Number of samples: 90
sport:
 Top cluster number: 1, Number of samples: 112
entertainment:
 Top cluster number: 1, Number of samples: 88

Map cluster number to category:


{1: 'entertainment', 2: 'tech'}

In [ ]:
data_train['clustered_category'] = data_train['cluster'].map(cluster_to_category)

In [ ]:
print('Overall accuracy of clustered categories:', np.mean(data_train['Category'] == data_train['clustered_category']))

Overall accuracy of clustered categories: 0.10201342281879194


In [ ]:
from sklearn.metrics import silhouette_score
print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(features,data_train['cluster']))

Silhouette Coefficient: 0.000


#Experiment with PCA

In [ ]:
data_embbedding=features
x=[]
y=[]
for i in range(1490,0,-20):
  pca_2 = PCA(n_components=i)
  pca_2_result = pca_2.fit_transform(data_embbedding)
  x.append(i)
  y.append(round(np.sum(pca_2.explained_variance_ratio_),2))
  print('Cumulative variance explained by {} principal components: {:.2%}'.format(i,np.sum(pca_2.explained_variance_ratio_)))

Cumulative variance explained by 1490 principal components: 100.00%
Cumulative variance explained by 1470 principal components: 100.00%
Cumulative variance explained by 1450 principal components: 100.00%
Cumulative variance explained by 1430 principal components: 100.00%
Cumulative variance explained by 1410 principal components: 99.99%
Cumulative variance explained by 1390 principal components: 99.97%
Cumulative variance explained by 1370 principal components: 99.95%
Cumulative variance explained by 1350 principal components: 99.92%
Cumulative variance explained by 1330 principal components: 99.88%
Cumulative variance explained by 1310 principal components: 99.83%
Cumulative variance explained by 1290 principal components: 99.77%
Cumulative variance explained by 1270 principal components: 99.69%
Cumulative variance explained by 1250 principal components: 99.60%
Cumulative variance explained by 1230 principal components: 99.49%
Cumulative variance explained by 1210 principal components

In [ ]:
#95%,90%,85% covarience-->925,780,670
data_embbedding=features
pca_95 = PCA(n_components=925)
pca_95result = pca_95.fit_transform(data_embbedding)

pca_90 = PCA(n_components=780)
pca_90result = pca_95.fit_transform(data_embbedding)

pca_85 = PCA(n_components=670)
pca_85result = pca_85.fit_transform(data_embbedding)

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(5, n_init=10, random_state=42)
kmeans.fit(pca_95result)
data_train['cluster95'] = kmeans.labels_

kmeans = KMeans(5, n_init=10, random_state=42)
kmeans.fit(pca_90result)
data_train['cluster90'] = kmeans.labels_

kmeans = KMeans(5, n_init=10, random_state=42)
kmeans.fit(pca_85result)
data_train['cluster85'] = kmeans.labels_

In [ ]:
cluster_to_category95= {}
for cat in data_train['Category'].unique():
    mark = data_train['Category'] == cat
    top = data_train[mark]['cluster95'].value_counts().head(1)
    count = top.values[0]
    cluster = top.index[0]
    print(f"{cat}:\n Top cluster number: {cluster}, Number of samples: {count}")
    cluster_to_category95[cluster] = cat

print("\nMap cluster number to category:")
cluster_to_category95

business:
 Top cluster number: 1, Number of samples: 101
tech:
 Top cluster number: 1, Number of samples: 78
politics:
 Top cluster number: 1, Number of samples: 89
sport:
 Top cluster number: 1, Number of samples: 116
entertainment:
 Top cluster number: 1, Number of samples: 92

Map cluster number to category:


{1: 'entertainment'}

In [ ]:
data_train['clustered_category95'] = data_train['cluster95'].map(cluster_to_category95)

In [ ]:
print('Overall accuracy of clustered categories:', np.mean(data_train['Category'] == data_train['clustered_category95']))

Overall accuracy of clustered categories: 0.06174496644295302


In [ ]:
from sklearn.metrics import silhouette_score
print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(pca_95result,data_train['cluster95']))

Silhouette Coefficient: 0.001


In [ ]:
cluster_to_category90= {}
for cat in data_train['Category'].unique():
    mark = data_train['Category'] == cat
    top = data_train[mark]['cluster90'].value_counts().head(1)
    count = top.values[0]
    cluster = top.index[0]
    print(f"{cat}:\n Top cluster number: {cluster}, Number of samples: {count}")
    cluster_to_category90[cluster] = cat

print("\nMap cluster number to category:")
cluster_to_category90

business:
 Top cluster number: 1, Number of samples: 107
tech:
 Top cluster number: 1, Number of samples: 81
politics:
 Top cluster number: 1, Number of samples: 92
sport:
 Top cluster number: 1, Number of samples: 120
entertainment:
 Top cluster number: 1, Number of samples: 97

Map cluster number to category:


{1: 'entertainment'}

In [ ]:
data_train['clustered_category90'] = data_train['cluster90'].map(cluster_to_category90)

In [ ]:
print('Overall accuracy of clustered categories:', np.mean(data_train['Category'] == data_train['clustered_category90']))

Overall accuracy of clustered categories: 0.06510067114093959


In [ ]:
from sklearn.metrics import silhouette_score
print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(pca_90result,data_train['cluster90']))

Silhouette Coefficient: 0.001


In [ ]:
cluster_to_category85= {}
for cat in data_train['Category'].unique():
    mark = data_train['Category'] == cat
    top = data_train[mark]['cluster85'].value_counts().head(1)
    count = top.values[0]
    cluster = top.index[0]
    print(f"{cat}:\n Top cluster number: {cluster}, Number of samples: {count}")
    cluster_to_category85[cluster] = cat

print("\nMap cluster number to category:")
cluster_to_category85

business:
 Top cluster number: 1, Number of samples: 96
tech:
 Top cluster number: 1, Number of samples: 68
politics:
 Top cluster number: 1, Number of samples: 75
sport:
 Top cluster number: 1, Number of samples: 104
entertainment:
 Top cluster number: 1, Number of samples: 84

Map cluster number to category:


{1: 'entertainment'}

In [ ]:
data_train['clustered_category85'] = data_train['cluster85'].map(cluster_to_category85)

In [ ]:
print('Overall accuracy of clustered categories:', np.mean(data_train['Category'] == data_train['clustered_category85']))

Overall accuracy of clustered categories: 0.056375838926174496


In [ ]:
from sklearn.metrics import silhouette_score
print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(pca_85result,data_train['cluster85']))

Silhouette Coefficient: 0.001
